# Chapter.9.05 SVLoR for Several Samples analysis

PA 9.05에서는 PA 9.04에서 진행했던 SVLoR for Several Samples에 대해서<br>
분석을 해볼 것이다

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import basic_nodes as nodes

np.random.seed(0)
plt.style.use('seaborn')
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.labelsize'] = 30
mpl.rcParams['axes.titlesize'] = 40
mpl.rcParams['legend.fontsize'] = 30

### Step.1
PA 9.04에서 사용한 모듈들을 가져옵니다

In [ ]:
class Affine:
    def __init__(self):
        self._feature_dim = 1
        self._Th = None
        
        self.node_imp()
        self.random_initialization()

    def node_imp(self):
        self._node1 = nodes.mul_node()
        self._node2 = nodes.plus_node()
    
    def random_initialization(self):
        r_feature_dim = 1/np.sqrt(self._feature_dim)
        self._Th = np.random.uniform(low = -1 * r_feature_dim,
                                     high = r_feature_dim,
                                     size = (self._feature_dim + 1,1))
    
    def forward(self,X):
        self._Z1 = self._node1.forward(self._Th[1], X)
        self._Z2 = self._node2.forward(self._Th[0], self._Z1)

        return self._Z2
    
    def backward(self, dZ, lr):
        dTh0, dZ1 = self._node2.backward(dZ)
        dTh1, dX = self._node1.backward(dZ1)

        self._Th[1] = self._Th[1] - lr*np.sum(dTh1)
        self._Th[0] = self._Th[0] - lr*np.sum(dTh0)
    
    def get_Th(self):
        return self._Th 
class Sigmoid:
    def __init__(self):
        self._Pred = None
    
    def forward(self, Z):
        self._Pred = 1/(1 + np.exp(-1*Z))
        return self._Pred
    
    def backward(self, dPred):
        partial = self._Pred * (1 - self._Pred)
        dZ = dPred * partial
        return dZ
class BinaryCrossEntropy_Cost:
    def __init__(self):
        self._Y, self._Pred = None, None
        self._mean_node =  nodes.mean_node()
    
    def forward(self, Y, Pred):
        self._Y, self._Pred = Y, Pred
        Loss = -1*(Y*np.log(self._Pred) + (1-Y)*np.log(1-Pred))
        J = self._mean_node.forward(Loss)
        return J
    
    def backward(self):
        dLoss = self._mean_node.backward(1)
        dPred = dLoss * (self._Pred - self._Y)/(self._Pred*(1-self._Pred))
        return dPred
class SVLoR:
    def __init__(self):
        self._feature_dim = 1
        self._affine = Affine()
        self._sigmoid = Sigmoid()

    def forward(self, X):
        Z = self._affine.forward(X)
        Pred =  self._sigmoid.forward(Z)
        return Pred
    
    def backward(self, dPred, lr):
        dZ = self._sigmoid.backward(dPred)
        self._affine.backward(dZ, lr)
        
    def  get_Th(self):
        return self._affine.get_Th()
def result_tracker():
    global iter_idx, check_freq
    global th_accum, model

    if iter_idx % check_freq == 0:
        th_accum = np.hstack((th_accum, model.get_Th()))
        cost_list.append(J)
    iter_idx += 1
def result_visualizer():
    global th_accum, cost_list
    fig,ax = plt.subplots(2, 1, figsize = (30,10))
    fig.subplots_adjust(hspace = 0.3)
    ax[0].set_title(r'$\vec{\theta}$' + 'Update ')

    ax[0].plot(th_accum[1,:], label = r'$\theta_{1}$')
    ax[0].plot(th_accum[0,:], label = r'$\theta_{0}$')

    ax[0].legend()
    iter_ticks = np.linspace(0,th_accum.shape[1],10).astype(np.int)
    ax[0].set_xticks(iter_ticks)

    ax[1].set_title('Cost')
    ax[1].plot(cost_list)
    ax[1].set_xticks(iter_ticks)

    n_pred = 1000
    fig,ax = plt.subplots(figsize = (30,10))
    ax.set_title('Predictor Update')
    ax.scatter(data[:,1], data[:,-1])

    ax_idx_arr = np.linspace(0,len(cost_list)-1,n_pred).astype(np.int)
    cmap = cm.get_cmap('rainbow',lut = len(ax_idx_arr))

    x_pred = np.linspace(np.min(data[:,1]),np.max(data[:,1]),1000)
    for ax_cnt, ax_idx in enumerate(ax_idx_arr):
        z = th_accum[1, ax_idx] * x_pred + th_accum[0,ax_idx]
        a = 1/(1 + np.exp(-1 * z))
        ax.plot(x_pred, a, color = cmap(ax_cnt),alpha = 0.2)

    y_ticks = np.round(np.linspace(0, 1, 7),2)
    ax.set_yticks(y_ticks)
def get_data_batch(data, batch_idx):
    global n_batch, batch_size
    
    if batch_idx is n_batch -1:
        batch = data[batch_idx*batch_size: ]
    else:
        batch = data[batch_idx*batch_size : (batch_idx+1)*batch_size]
    return batch
def dataset_generator(x_dict):
    x_data = np.random.normal(x_dict['mean'], x_dict['std'],x_dict['n_sample'])
    x_data_noise = x_data + x_dict['noise_factor'] * np.random.normal(0,1,x_dict['n_sample'])
    
    if x_dict['direction'] > 0:
        y_data = (x_data_noise > x_dict['cutoff']).astype(np.int)
    else:
        y_data = (x_data_noise < x_dict['cutoff']).astype(np.int)
    
    data = np.zeros(shape = (x_dict['n_sample'],1))
    data = np.hstack((data,x_data.reshape(-1,1),y_data.reshape(-1,1)))
    return data

### Step.2
비교를 위하여 PA 9.04에서 했던 값을 동일하게 진행합니다<br>
"""<br>
mean: 0<br>
std: 1<br>
n_sample: 100<br>
noise_factor: 0.5<br>
cutoff: 0<br>
direction: 1<br>
batch_size: 8<br>
epochs: 500<br>
lr = 0.05<br>
iter_idx :0<br>
check_freq: 5<br>
"""<br><br>

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = dataset_generator(x_dict)
batch_size =  
n_batch =

model =  
BCE_Cost = 

th_accum =
cost_list = []
epochs, lr =  
iter_idx, check_freq = 

for epoch in range(epochs):
    np.random.shuffle(data)
    
    for batch_idx in range(n_batch):
        batch =
        X,Y =  

        Pred =
        J = 

        dPred =
        model.backward(dPred, lr)
### End
        result_tracker()
result_visualizer()

**Expected Output** 
<img src='./img_9/9_15.png' width = 700>
<img src='./img_9/9_16.png' width = 700>

### Step.3
먼저 mean을 증가시켜서 학습 결과를 확인합니다<br>
mean:10<br>
std:1<br>
cutoff:10<br>
epochs:500<br>
learning rate:0.05

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = dataset_generator(x_dict)

batch_size = 
n_batch = 

model =  
BCE_Cost = 

th_accum = 
cost_list = []
epochs, lr =  
iter_idx, check_freq = 



for epoch in range(epochs):
    np.random.shuffle(data)
    
    for batch_idx in range(n_batch):
        batch = 
        X,Y =  

        Pred =
        J = 

        dPred =
        model.backward(dPred, lr)
### End
        result_tracker()
result_visualizer()

**Expected Output** 
<img src='./img_9/9_17.png' width = 700>
<img src='./img_9/9_18.png' width = 700>

mean의 값을 증가시켜서 학습하게 되면<br>
x2와 x1이 불균형하게 학습되는 모습을 확인할 수 있다

### Step.4
먼저 mean을 증가시켜서 학습 결과를 확인합니다<br>
mean:1<br>
std:7<br>
cutoff:1<br>
epochs:500<br>
learning rate:0.0005

In [ ]:
### Start
x_dict =  {'mean': , 'std': , 'n_sample': ,
            'noise_factor': ,
            'cutoff': , 'direction': }

data = dataset_generator(x_dict)

batch_size = 
n_batch = 

model =  
BCE_Cost = 

th_accum = 
cost_list = []
epochs, lr =  
iter_idx, check_freq = 



for epoch in range(epochs):
    np.random.shuffle(data)
    
    for batch_idx in range(n_batch):
        batch =
        X,Y =

        Pred = 
        J =

        dPred = 
        model.backward(dPred, lr)

        result_tracker()
result_visualizer()
### End

**Expected Output** 
<img src='./img_9/9_19.png' width = 700>
<img src='./img_9/9_20.png' width = 700>

x1와 x0가 불균형하게 학습되는 모습을 볼수 있지만<br>
Step.3 mean을 변경했을때보다는 비교적 안정적으로<br>
학습되는 모습을 확인할 수 있다